# Step 4: Machine Learning Evaluation for Slowloris vs. Benign Traffic

This notebook:
1. Loads the preprocessed dataset.
2. Filters to Slowloris vs. Benign samples.
3. Defines candidate features and binary target.
4. Standardizes features and splits into training/test.
5. Applies stratified sampling to speed up training.
6. Trains and evaluates MLP, SVM, Random Forest, Logistic Regression, and XGBoost.
7. Displays performance summary, confusion matrices, and ROC curves.

## 1. Imports

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, confusion_matrix, roc_curve, auc)

## 2. Load and Filter Data

In [4]:
# Load preprocessed dataset
data = pd.read_csv("preprocessed_CICIoT2023.csv")
print("Original data shape:", data.shape)

# Filter to the two classes
labels_of_interest = ["DDoS-SlowLoris", "BenignTraffic"]
data = data[data["label"].isin(labels_of_interest)].reset_index(drop=True)
print("Filtered data shape:", data.shape)
print("Labels present:", data["label"].unique())

Original data shape: (46686545, 47)
Filtered data shape: (1121621, 47)
Labels present: ['BenignTraffic' 'DDoS-SlowLoris']


## 3. Feature Selection and Target Creation

In [ ]:
# Candidate features
candidate_features = [
    "flow_duration", "Rate", "Srate", "IAT",
    "ack_flag_number", "Magnitue", "Radius",
    "Variance", "Number", "Weight"
]

# Check for missing features
missing = [f for f in candidate_features if f not in data.columns]
if missing:
    print("Warning: Missing features:", missing)

# Create binary target (1 = Slowloris, 0 = Benign)
data['target'] = (data['label'] == "DDoS-SlowLoris").astype(int)

X = data[candidate_features]
y = data['target']
print("Feature matrix shape:", X.shape)
print("Target distribution:\n", y.value_counts())

KeyError: "['Magnitude'] not in index"

## 4. Standardize & Split

In [ ]:
# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)
print("Train/test shapes:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

## 5. Stratified Sampling (Optional)

In [ ]:
# Reduce training set to 10% for faster training
sss = StratifiedShuffleSplit(n_splits=1, train_size=0.1, random_state=42)
for train_idx, _ in sss.split(X_train, y_train):
    X_train_small = X_train[train_idx]
    y_train_small = y_train.iloc[train_idx]

print("Sampled train shape:", X_train_small.shape)

## 6. Train & Evaluate Models

In [ ]:
# Initialize models
models = {
    "MLP": MLPClassifier(random_state=42, max_iter=500),
    "SVM": SVC(probability=True, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=500, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# Evaluate
results = {}
for name, model in models.items():
    model.fit(X_train_small, y_train_small)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:,1]
    results[name] = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, zero_division=0),
        "Recall": recall_score(y_test, y_pred, zero_division=0),
        "F1-Score": f1_score(y_test, y_pred, zero_division=0),
        "ROC-AUC": roc_auc_score(y_test, y_proba)
    }

# Summary DataFrame
results_df = pd.DataFrame(results).T
print("### Model Performance Summary ###")
display(results_df)

In [ ]:
# Performance Bar Chart using 'plasma' palette
models_list = results_df.index.tolist()
metrics = results_df.columns.tolist()
colors = sns.color_palette("plasma", len(metrics))
palette = dict(zip(metrics, colors))

x = np.arange(len(models_list))
width = 0.15
fig, ax = plt.subplots(figsize=(12,6), layout='constrained')

for i, metric in enumerate(metrics):
    vals = results_df[metric].values
    rects = ax.bar(x + i*width, vals, width, label=metric, color=palette[metric])
    for bar in rects:
        ax.text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.01,
                f"{bar.get_height():.3f}", ha='center', va='bottom', rotation=90, fontsize=8)

ax.set_xticks(x + width*(len(metrics)-1)/2)
ax.set_xticklabels(models_list)
ax.set_ylabel("Score")
ax.set_title("Model Performance Metrics")
ax.legend(title="Metric", ncols=len(metrics), loc='upper left')
plt.ylim(0,1.1)
plt.show()

In [ ]:
# 7. Confusion Matrices
class_labels = ["Benign", "Slowloris"]
for name, model in models.items():
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_labels, yticklabels=class_labels)
    plt.title(f"Confusion Matrix: {name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

In [ ]:
# 8. ROC Curves
model_order = results_df.index.tolist()
colors_model = sns.color_palette("Dark2", len(model_order))
roc_palette = dict(zip(model_order, colors_model))

plt.figure(figsize=(8,6))
for name in model_order:
    model = models[name]
    y_proba = model.predict_proba(X_test)[:,1]
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{name} (AUC={roc_auc:.3f})",
             color=roc_palette[name], linewidth=2)

plt.plot([0,1],[0,1],'k--',linewidth=1)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves for All Models")
plt.legend(loc="lower right")
plt.show()

**Conclusion:**

- The performance bar chart highlights comparative strengths across metrics.
- Confusion matrices reveal class‐specific errors.
- ROC curves illustrate model discrimination ability.